# Fine tuning models
All notebooks for finetuning models were the same, only the model changed. This notebook shows how we fine-tuned ```flax-community/roberta-base-danish``` but could just as have been for any other model.

The notebook requires GPU access, otherwise it will either be very slow or the script will crash. 


# GPU, installing packages and login to WANDB

In [ ]:
# Initialize GPU
!nvidia-smi

Mon Nov 28 10:16:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installing required packages

In [ ]:
!pip install -q transformers transformers-interpret datasets evaluate wandb tensorflow spacy spacy_langdetect

     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 45 kB 4.3 MB/s 
     |████████████████████████████████| 451 kB 89.4 MB/s 
     |████████████████████████████████| 72 kB 1.7 MB/s 
     |████████████████████████████████| 1.9 MB 62.5 MB/s 
     |████████████████████████████████| 182 kB 84.3 MB/s 
     |████████████████████████████████| 7.6 MB 52.3 MB/s 
     |████████████████████████████████| 1.4 MB 73.2 MB/s 
     |████████████████████████████████| 793 kB 84.3 MB/s 
     |████████████████████████████████| 1.6 MB 63.2 MB/s 
     |████████████████████████████████| 115 kB 86.2 MB/s 
     |████████████████████████████████| 212 kB 90.4 MB/s 
     |████████████████████████████████| 127 kB 80.2 MB/s 
     |████████████████████████████████| 168 kB 89.2 MB/s 
     |████████████████████████████████| 182 kB 75.6 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 168 kB 88.2 MB/s 
     |███████████████

# Importing packages, data and model

In [ ]:
from datasets import load_dataset, load_metric #load_dataset will cache the dataset to avoid downloading it again the next time you run this cell.
import datasets as datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import pandas as pd

# Loading data
dataset_1 = datasets.load_dataset('ScandEval/angry-tweets-mini', split='train+test+val')# sentiment classification dataset: negative, neutral or positive

# Loading larger dataset
dataset_2 = load_dataset("DDSC/twitter-sent", split='train+test')


# Loading model and tokenizer
model_name = "flax-community/roberta-base-danish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_labels=3
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels) # giving the number of labels and huggingface path,beware that "AutoModelForSequenceClassification" will automatically add an empty linear layer on top of the model, we don't need to do that manually

Some weights of the model checkpoint at flax-community/roberta-base-danish were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at flax-community/roberta-base-danish and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight',

In [ ]:
from datasets import concatenate_datasets
raw_dataset = concatenate_datasets([dataset_1, dataset_2])

In [ ]:
from datasets import dataset_dict, Dataset

pandas = pd.DataFrame(raw_dataset)

pandas.loc[pandas.label == 'negativ', 'label'] = "negative"
pandas.loc[pandas.label == 'positiv', 'label'] = "positive"

# Pre-processing data

**Removing duplicate rows**

In [ ]:
# removing duplicate rows
pandas = pandas.drop_duplicates().reset_index()
pandas = pandas.drop(['index'], axis = 1)

**Removing specific words**

In [ ]:
def remove_mystopwords(sentence, stopword_list):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in stopword_list]
    return (" ").join(tokens_filtered)

# creating stopword list
stopwords = ["link", "rt", "amp", "@USER", "[LINK]"]

pandas.text = [remove_mystopwords(sentence, stopwords) for sentence in pandas.text]

**Removing non-Danish sentences**

In [ ]:
no_eng = 1
no_below_4 = 1
no_ttr_below_3 = 1

if no_eng == 1:
  import spacy
  from spacy.language import Language
  from spacy_langdetect import LanguageDetector
  import numpy as np
  import pandas as pd

  def get_lang_detector(nlp, name):
    return LanguageDetector()
    
  # loading the language model instance that will be used for language detection
  nlp = spacy.load("en_core_web_sm")
  Language.factory("language_detector", func=get_lang_detector)
  nlp.add_pipe('language_detector', last=True)

  # applying the language detection to the data
  data = [nlp(text_i)._.language for i, text_i in enumerate(pandas['text'])]

  # transforming the data to a pandas dataframe
  data_pd = pd.DataFrame.from_dict(data)
  data_pd["tweets"] = pandas['text'] # adding the tweets to the dataframe

  # removing all that have been detected to be english
 # data_pd_1 = data_pd[data_pd['language'] != 'en']

**Removing sentences of length <4 words**

In [ ]:
data_pd['label'] = pandas['label']

if no_below_4 == 1:

  # removing all with a sentence length below 4
  data_pd['tweet_len'] = [len(data_pd['tweets'].iloc[i].split()) for i in range(data_pd.shape[0])]
  data_pd = data_pd[data_pd['tweet_len'] > 3]

data_pd_3 = data_pd[data_pd['language'] != 'en']

data_pd_4 = data_pd_3.drop(['score', 'language', 'tweet_len'], axis= 1)
data_pd_5 = data_pd_4.rename(columns = {'tweets': 'text'})

In [ ]:
raw_dataset = Dataset.from_dict(data_pd_5)
raw_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 3806
})

**Changing predefined dataset splits**

In [ ]:
from datasets import concatenate_datasets, dataset_dict, Dataset

#### combine datasets
#dataset_combined = concatenate_datasets([raw_dataset['val'], raw_dataset['train'], raw_dataset['test']])

##### 60% train, 40% test
train_test = raw_dataset.train_test_split(test_size=0.4, seed = 42) # seed when splitting data is fairly crucial when comparing different models, to make sure they get the same test and training data.
# 20% validation, 20% test
test_valid = train_test['test'].train_test_split(test_size=0.5, seed = 42)
# combining into test 60%, test 20%, val 20%
dataset_recombined = datasets.DatasetDict({
    'train': train_test['train'],
    'valid': test_valid['train'],
    'test': test_valid['test']})

In [ ]:
# creating a ClassLabel instance to use for mapping classes to integers (it is needed for creating tensors)
from datasets import ClassLabel
labels_cl = ClassLabel(num_classes=3, names=['negative', 'neutral', 'positive'])

# defining a function to tokenize the text and translate all labels into integers intead of strings
def tokenize_function(example):
  tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
  tokens['label'] = labels_cl.str2int(example['label'])
  return tokens

tokenized_datasets = dataset_recombined.map(tokenize_function, batched=True, remove_columns=dataset_recombined['train'].column_names) # batched=True speeds up tokenization by allowing to process multiple lines at once

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Evaluation metrics

In [ ]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    metric0 = evaluate.load("accuracy")
    metric1 = evaluate.load("precision")
    metric2 = evaluate.load("recall")
    metric3 = evaluate.load("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric0.compute(predictions=predictions, references=labels)["accuracy"]
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

## Early stopping

In [ ]:
# Makes the model stop when validation loss hasn't improved for n(early_stopping_patience) epochs
early_stop = EarlyStoppingCallback(early_stopping_patience = 8)

 ## Defining hyperparameters 

In [ ]:
batch_size = 128 # stating batch size
epochs = 200
learning_rate = 2e-5

## WANDB

In [ ]:
import wandb
wandb.login()
wandb.init(project="bachelor_thesis_cogsci",
           tags=["HPsearch_nbailab"])

wandb.config.dropout = 0.2

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jorgenhw (bachelor_thesis_cogsci). Use `wandb login --relogin` to force relogin


## Hyperparameter tuning

In [ ]:
!pip install -q optuna ray[tune]

     |████████████████████████████████| 348 kB 4.4 MB/s 
     |████████████████████████████████| 59.1 MB 107.1 MB/s 
     |████████████████████████████████| 81 kB 12.6 MB/s 
     |████████████████████████████████| 209 kB 92.8 MB/s 
     |████████████████████████████████| 78 kB 9.4 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 147 kB 90.7 MB/s 
     |████████████████████████████████| 112 kB 63.0 MB/s 
     |████████████████████████████████| 8.8 MB 71.8 MB/s 
     |████████████████████████████████| 125 kB 84.1 MB/s 
     |████████████████████████████████| 468 kB 62.4 MB/s 


In [ ]:
def model_init():
    return model #model = AutoModelForSequenceClassification.from_pretrained(model, num_labels=3) (defined earlier)

**Training parameters**

In [ ]:
training_args = TrainingArguments(output_dir=model_name, 
                                  evaluation_strategy = "epoch",
                                  save_strategy = "epoch", 
                                  num_train_epochs = epochs, 
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  learning_rate = learning_rate,
                                  weight_decay=0.01,
                                  load_best_model_at_end=True,
                                  report_to="wandb",
                                  save_total_limit = 2)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [early_stop]
)

In [ ]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_loguniform('weight_decay', 1e-4, 1e-2)
    }

# Initializing hyperparameter tuning

In [ ]:
import optuna
from optuna.samplers import TPESampler


sampler = optuna.samplers.TPESampler()
pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)



best_run = trainer.hyperparameter_search(
    n_trials=10, 
    direction="minimize", 
    hp_space=my_hp_space, 
    backend = "optuna",
    sampler = sampler,
    pruner = pruner
    )

[I 2022-11-28 10:24:12,072] A new study created in memory with name: no-name-17663d9a-56ae-44a0-abab-db7bc77e939c
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  after removing the cwd from sys.path.
Trial: {'learning_rate': 1.0211178508623191e-06, 'weight_decay': 0.0035668865310456033}
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2283
  Num Epochs = 200
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & a

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.125018,0.347769,0.359674,0.347769,0.352620
2,No log,1.102760,0.421260,0.412319,0.421260,0.412080
3,No log,1.089727,0.425197,0.409788,0.425197,0.399620
4,No log,1.077516,0.427822,0.420304,0.427822,0.391176
5,No log,1.072517,0.433071,0.426764,0.433071,0.383327
6,No log,1.060710,0.446194,0.438701,0.446194,0.397581
7,No log,1.057013,0.446194,0.438887,0.446194,0.400846
8,No log,1.056450,0.429134,0.429545,0.429134,0.393532
9,No log,1.040166,0.459318,0.452581,0.459318,0.420635
10,No log,1.024630,0.475066,0.471017,0.475066,0.433427


***** Running Evaluation *****
  Num examples = 762
  Batch size = 128


...

***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/run-8/checkpoint-18
Configuration saved in NbAiLab/nb-bert-large/run-8/checkpoint-18/config.json
Model weights saved in NbAiLab/nb-bert-large/run-8/checkpoint-18/pytorch_model.bin
tokenizer config file saved in NbAiLab/nb-bert-large/run-8/checkpoint-18/tokenizer_config.json
Special tokens file saved in NbAiLab/nb-bert-large/run-8/checkpoint-18/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/run-8/checkpoint-36
Configuration saved in NbAiLab/nb-bert-large/run-8/checkpoint-36/config.json
Model weights saved in NbAiLab/nb-bert-large/run-8/checkpoint-36/pytorch_model.bin
tokenizer config file saved in NbAiLab/nb-bert-large/run-8/checkpoint-36/tokenizer_config.json
Special tokens file saved in NbAiLab/nb-bert-large/run-8/checkpoint-36/special_tokens_map.json
***** Running Evalua

eval/accuracy,▂▆▃▅▆▆▁█▄
eval/f1,▄▅▃▄▆▆▂█▁
eval/loss,▁▁▂▃▃▃▅▅█
eval/precision,▅▅▁▂▇▅█▄▃
eval/recall,▂▆▃▅▆▆▁█▄
eval/runtime,▅▄▁▁█▃▅▇▁
eval/samples_per_second,▄▅██▁▆▄▂█
eval/steps_per_second,▄▅██▁▆▄▂▇
train/epoch,▁▂▃▄▅▅▆▇██
train/global_step,▁▂▃▄▅▅▆▇██
train/total_flos,▁


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.210971,0.653543,0.663371,0.653543,0.654579
2,No log,1.044037,0.681102,0.695460,0.681102,0.683973
3,No log,1.123331,0.698163,0.701714,0.698163,0.698936
4,No log,1.363359,0.685039,0.686160,0.685039,0.676897
5,No log,1.325615,0.641732,0.676949,0.641732,0.644387
6,No log,1.509836,0.681102,0.679322,0.681102,0.678639
7,No log,1.484062,0.690289,0.694544,0.690289,0.682628
8,No log,1.476684,0.687664,0.686338,0.687664,0.686116
9,No log,1.563144,0.669291,0.675941,0.669291,0.671290
10,No log,1.726755,0.620735,0.654785,0.620735,0.623568


***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/run-9/checkpoint-18
Configuration saved in NbAiLab/nb-bert-large/run-9/checkpoint-18/config.json
Model weights saved in NbAiLab/nb-bert-large/run-9/checkpoint-18/pytorch_model.bin
tokenizer config file saved in NbAiLab/nb-bert-large/run-9/checkpoint-18/tokenizer_config.json
Special tokens file saved in NbAiLab/nb-bert-large/run-9/checkpoint-18/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/run-9/checkpoint-36
Configuration saved in NbAiLab/nb-bert-large/run-9/checkpoint-36/config.json
Model weights saved in NbAiLab/nb-bert-large/run-9/checkpoint-36/pytorch_model.bin
tokenizer config file saved in NbAiLab/nb-bert-large/run-9/checkpoint-36/tokenizer_config.json
Special tokens file saved in NbAiLab/nb-bert-large/run-9/checkpoint-36/special_tokens_map.json
***** Running Evalua

In [ ]:
# calling best run
best_run

BestRun(run_id='0', objective=2.595929793216958, hyperparameters={'learning_rate': 1.0211178508623191e-06, 'weight_decay': 0.0035668865310456033})

# Initializing fine-tuning with parameters from hyperparameter search
The below loop trains the model 10 times and saves the results from each training. This is due to langauge models being stochastic, so taking the average performance of 10 runs gives a more accurate estimate of performance.

In [ ]:
for i in range(10):
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["test"],
      compute_metrics=compute_metrics,
      callbacks = [early_stop])

  for n, v in best_run.hyperparameters.items():
      setattr(trainer.args, n, v) # for running the experiment with the best hyperparameters from the hyperparameters search

  trainer.train() # argument trial can be used for hyperparameter search

  trainer.evaluate()

  import tensorflow as tf

  # creating model predictions for the validation data
  predictions_val = trainer.predict(tokenized_datasets["valid"])

  # choosing the prediction that has the highest probability 
  preds_val_val = np.argmax(predictions_val.predictions, axis=-1)

  # calculating the probabilities instead of logits from each
  predictions_probabilities = tf.nn.softmax(predictions_val.predictions)

  def compute_metrics_end(preds, refs):
      metric0 = evaluate.load("accuracy")
      metric1 = evaluate.load("precision")
      metric2 = evaluate.load("recall")
      metric3 = evaluate.load("f1")
      
      #logits, labels = eval_pred
      #predictions = np.argmax(logits, axis=-1)
      accuracy = metric0.compute(predictions=preds, references=refs)["accuracy"]
      precision = metric1.compute(predictions=preds, references=refs, average="weighted")["precision"]
      recall = metric2.compute(predictions=preds, references=refs, average="weighted")["recall"]
      f1 = metric3.compute(predictions=preds, references=refs, average="weighted")["f1"]
      return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

  metrics_val = compute_metrics_end(preds=preds_val_val, refs=predictions_val.label_ids)

  import tensorflow as tf

  # creating model predictions for the validation data
  predictions_test = trainer.predict(tokenized_datasets["test"])

  # choosing the prediction that has the highest probability 
  preds_test_test = np.argmax(predictions_test.predictions, axis=-1)

  # calculating the probabilities instead of logits from each
  predictions_probabilities_test = tf.nn.softmax(predictions_test.predictions)

  metrics_test = compute_metrics_end(preds=preds_test_test, refs=predictions_test.label_ids)

  import pandas as pd

  data = {'Predicted Labels': ["negative" if i == 0 else "neutral" if i == 1 else "positive" for i in preds_val_val],
          'True Labels': ["negative" if i == 0 else "neutral" if i == 1 else "positive" for i in predictions_val.label_ids],
          'Misclassification': ["TRUE" if preds_val_val[i] == predictions_val.label_ids[i] else 'MISS' for i, val in enumerate(preds_val_val)],
          'Text': dataset_recombined['valid']['text'],
          'Logit Values': [str(i) for i in predictions_val.predictions],
          'Probabilities': [str(i) for i in np.asarray(predictions_probabilities)]}
  df = pd.DataFrame(data)
  df_metrics_val = pd.DataFrame(metrics_val.items())
  df_metrics_test = pd.DataFrame(metrics_test.items())

  df.to_csv(f"/content/drive/MyDrive/BA_data/nbailab/df_classification_report{i}.csv")
  df_metrics_val.to_csv(f"/content/drive/MyDrive/BA_data/nbailab/df_classification_metrics_val{i}.csv")
  df_metrics_test.to_csv(f"/content/drive/MyDrive/BA_data/nbailab/df_classification_metrics_test{i}.csv")

wandb.finish()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2283
  Num Epochs = 200
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 3600
  Number of trainable parameters = 355090435
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.616209,0.658793,0.670291,0.658793,0.661832
2,No log,1.611414,0.671916,0.673999,0.671916,0.672767
3,No log,1.638412,0.671916,0.671111,0.671916,0.671282
4,No log,1.674264,0.671916,0.671023,0.671916,0.670918
5,No log,1.703730,0.675853,0.675233,0.675853,0.675121
6,No log,1.726892,0.674541,0.673049,0.674541,0.673514
7,No log,1.745444,0.677165,0.675593,0.677165,0.676094
8,No log,1.774629,0.677165,0.675461,0.677165,0.675948
9,No log,1.793698,0.675853,0.674472,0.675853,0.674952
10,No log,1.819797,0.669291,0.668465,0.669291,0.668685


***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/checkpoint-18
Configuration saved in NbAiLab/nb-bert-large/checkpoint-18/config.json
Model weights saved in NbAiLab/nb-bert-large/checkpoint-18/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/checkpoint-36
Configuration saved in NbAiLab/nb-bert-large/checkpoint-36/config.json
Model weights saved in NbAiLab/nb-bert-large/checkpoint-36/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/checkpoint-54
Configuration saved in NbAiLab/nb-bert-large/checkpoint-54/config.json
Model weights saved in NbAiLab/nb-bert-large/checkpoint-54/pytorch_model.bin
Deleting older checkpoint [NbAiLab/nb-bert-large/checkpoint-18] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 762
  Batch s

***** Running Prediction *****
  Num examples = 761
  Batch size = 128
***** Running Prediction *****
  Num examples = 762
  Batch size = 128


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2283
  Num Epochs = 200
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 3600
  Number of trainable parameters = 355090435
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


...

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2283
  Num Epochs = 200
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 3600
  Number of trainable parameters = 355090435
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.993423,0.682415,0.683739,0.682415,0.682954
2,No log,2.010260,0.678478,0.679380,0.678478,0.678869
3,No log,2.031716,0.666667,0.666418,0.666667,0.666395
4,No log,2.030780,0.664042,0.663085,0.664042,0.663097
5,No log,2.047827,0.664042,0.664280,0.664042,0.663979
6,No log,2.047661,0.670604,0.671228,0.670604,0.670892
7,No log,2.043708,0.678478,0.680406,0.678478,0.679259
8,No log,2.049685,0.678478,0.679109,0.678478,0.678745
9,No log,2.070817,0.674541,0.675105,0.674541,0.674809


***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/checkpoint-18
Configuration saved in NbAiLab/nb-bert-large/checkpoint-18/config.json
Model weights saved in NbAiLab/nb-bert-large/checkpoint-18/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/checkpoint-36
Configuration saved in NbAiLab/nb-bert-large/checkpoint-36/config.json
Model weights saved in NbAiLab/nb-bert-large/checkpoint-36/pytorch_model.bin
Deleting older checkpoint [NbAiLab/nb-bert-large/checkpoint-162] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 762
  Batch size = 128
Saving model checkpoint to NbAiLab/nb-bert-large/checkpoint-54
Configuration saved in NbAiLab/nb-bert-large/checkpoint-54/config.json
Model weights saved in NbAiLab/nb-bert-large/checkpoint-54/pytorch_model.bin
Deleting older checkpoint [NbAiLab/nb-bert-large/checkpoint-

***** Running Prediction *****
  Num examples = 761
  Batch size = 128
***** Running Prediction *****
  Num examples = 762
  Batch size = 128


eval/accuracy,▁█▅▃▄▅▅▃▄▄▅▅▄▅▅▃▃▅▄▄▄▅▃▃▅▄▃▄▄▅▃▅▅▃▃▄▆▃▅▆
eval/f1,▁█▅▄▄▄▄▃▄▄▄▅▄▄▄▃▃▅▄▄▃▄▃▃▅▄▃▄▄▅▃▄▅▃▃▄▅▂▅▅
eval/loss,▂▁▄▄▅▅▆▆▅▆▆▅▆▆▇▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇█▇▇███████
eval/precision,▁█▄▃▃▃▃▂▃▂▃▄▃▃▃▁▂▄▃▃▂▃▂▂▃▃▂▃▃▄▂▃▄▂▂▃▅▁▄▅
eval/recall,▁█▅▃▄▅▅▃▄▄▅▅▄▅▅▃▃▅▄▄▄▅▃▃▅▄▃▄▄▅▃▅▅▃▃▄▆▃▅▆
eval/runtime,▁▃▂▃▃▃▂▃▂▂▂▁▂▁▂▁▁▁▄▂▁▂▂█▁▂▃▂▁▃▁▂▃▂▂▂▄▂▂▄
eval/samples_per_second,█▆▇▆▆▆▇▆▇▇▇█▇█▇███▅▇█▇▇▁█▇▆▇█▆█▇▆▇▇▇▅▇▇▅
eval/steps_per_second,█▆▇▆▆▆▇▆▇▇▇█▇█▇███▅▇█▇▇▁█▇▆▇█▆█▇▆▇▇▇▅▇▇▅
train/epoch,▁▃▆█▃▅▇█▃▅▇▁▃▆▇▃▅▇▁▃▆▇▂▄▆▇▃▆▇▂▄▆▇▃▅▇▁▃▆▇
train/global_step,▁▃▆█▃▅▇█▃▅▇▁▃▆▇▃▅▇▁▃▆▇▂▄▆▇▃▆▇▂▄▆▇▃▅▇▁▃▆▇
train/total_flos,█▁▁▁▁▁▁▁▁▁
